In [3]:
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.properties import iapws95
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_lmtd_callback


ImportError: cannot import name 'BlockData' from 'pyomo.core.base.block' (c:\Users\Sara\miniforge3\envs\my-idaes-env\lib\site-packages\pyomo\core\base\block.py)

In [18]:

# Create a Pyomo model
model = pyo.ConcreteModel()

# Create a flowsheet block in the model
model.fs = FlowsheetBlock(dynamic=False)

# Set up the property package (IAPWS95 for water properties in this case)
model.fs.properties = iapws95.Iapws95ParameterBlock()


In [1]:

# Add a hx model to the flowsheet.
model.fs.H101 = HeatExchanger(
    delta_temperature_callback=delta_temperature_lmtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

NameError: name 'HeatExchanger' is not defined

In [23]:

# Set some example conditions for the heat exchanger (inlet flow rates, temperatures, etc.)
# Example: Set up inlet and outlet conditions for hot side (shell) and cold side (tube)
model.fs.H101.shell.inlet.flow_mol[0].fix(10)  # Hot fluid flow in mol/s (shell side)


AttributeError: '_ScalarControlVolume0DBlock' object has no attribute 'inlet'

In [ ]:

model.fs.H101.shell.inlet.temperature[0].fix(662.30+273.15)  # Inlet temperature (K)
model.fs.H101.tube.inlet.flow_mol[0].fix(10)  # Cold fluid flow in mol/s (tube side)
model.fs.H101.tube.inlet.temperature[0].fix(20.00+273.15)  # Inlet temperature (K)


In [ ]:


# Fix the overall heat transfer coefficient (U)
m.fs.H102.U.fix(30)  # Fixed overall heat transfer coefficient [W/m²·K]

# Initialize the model (this is an essential step before solving)
m.fs.H101.initialize()

# Solve the model to calculate the area
solver = pyo.SolverFactory('ipopt')
solver.options = {'tol': 1e-6}
results = solver.solve(m, tee=True)

# Extract the calculated area from the heat exchanger model
calculated_area = m.fs.H101.area

# Print the calculated area
print(f"Calculated Heat Exchanger Area: {calculated_area.value} m²")
